In [1]:
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;

public class Cadet {
    public string Name { get; set; }
    public string Group { get; set; }
    public string Discipline { get; set; }
    public int Mark { get; set; }
}

public class CadetGPA {
    public string Cadet { get; set; }
    public double GPA { get; set; }
}

In [2]:
public void GetStudentsWithHighestGPA(string inputPath, string outputPath) {
    // Read input file
    string inputJson = File.ReadAllText(inputPath);

    // Parse input JSON
    dynamic inputData = JsonConvert.DeserializeObject(inputJson);
    List<Cadet> cadets = JsonConvert.DeserializeObject<List<Cadet>>(inputData.data.ToString());

    // Calculate GPA for each cadet
    var cadetGpas = cadets.GroupBy(c => c.Name)
        .Select(g => new CadetGPA
        {
            Cadet = g.Key,
            GPA = g.Average(c => c.Mark)
        })
        .ToList();

    // Find cadets with the highest GPA
    double maxGpa = cadetGpas.Max(c => c.GPA);
    List<CadetGPA> highestGpaCadets = cadetGpas.Where(c => c.GPA == maxGpa).ToList();

    // Create output data
    var outputData = new
    {
        Response = highestGpaCadets
    };

    // Write output file
    string outputJson = JsonConvert.SerializeObject(outputData, Formatting.Indented);
    File.WriteAllText(outputPath, outputJson);
}

In [3]:
public void CalculateGPAByDiscipline(string inputPath, string outputPath) {
    // Read input file
    string inputJson = File.ReadAllText(inputPath);

    // Parse input JSON
    dynamic inputData = JsonConvert.DeserializeObject(inputJson);
    List<Cadet> cadets = JsonConvert.DeserializeObject<List<Cadet>>(inputData.data.ToString());

    // Calculate GPA for each discipline
    var disciplineGpas = cadets.GroupBy(c => c.Discipline)
        .Select(g =>
            new JObject(
                new JProperty(g.Key, g.Average(c => c.Mark))
            )
        ).ToList();

    // Create output data
    var outputData = new
    {
        Response = disciplineGpas
    };

    // Write output file
    string outputJson = JsonConvert.SerializeObject(outputData, Formatting.Indented);
    File.WriteAllText(outputPath, outputJson);
}

In [4]:
public void GetBestGroupsByDiscipline(string inputPath, string outputPath) {
    // Read input file
    string inputJson = File.ReadAllText(inputPath);

    // Parse input JSON
    dynamic inputData = JsonConvert.DeserializeObject(inputJson);
    List<Cadet> cadets = JsonConvert.DeserializeObject<List<Cadet>>(inputData.data.ToString());

    // Calculate GPA for each group in each discipline
    var groupGpas = cadets.GroupBy(c => new { c.Discipline, c.Group })
        .Select(g => new
        {
            Discipline = g.Key.Discipline,
            Group = g.Key.Group,
            GPA = g.Average(c => c.Mark)
        })
        .ToList();

    // Find the best group for each discipline
    var bestGroups = groupGpas.GroupBy(g => g.Discipline)
        .Select(d => d.OrderByDescending(g => g.GPA).First())
        .ToList();

    // Create output data
    var outputData = new
    {
        Response = bestGroups
    };

    // Write output file
    string outputJson = JsonConvert.SerializeObject(outputData, Formatting.Indented);
    File.WriteAllText(outputPath, outputJson);
}

In [5]:
public void ProcessFile(string filename_in, string filename_out) {
    JObject json = JObject.Parse(File.ReadAllText($@"./files/{filename_in}"));

    string taskName = (string)json["taskName"];

    if (taskName == "GetStudentsWithHighestGPA") {
        GetStudentsWithHighestGPA($"./files/{filename_in}", $"./files/{filename_out}");
    } else if (taskName == "CalculateGPAByDiscipline") {
        CalculateGPAByDiscipline($"./files/{filename_in}", $"./files/{filename_out}");
    } else if (taskName == "GetBestGroupsByDiscipline") {
        GetBestGroupsByDiscipline($"./files/{filename_in}", $"./files/{filename_out}");
    }
}

ProcessFile("GetStudentsWithHighestGPA.json", "GetStudentsWithHighestGPA_out.json");
ProcessFile("GetBestGroupsByDiscipline.json", "GetBestGroupsByDiscipline_out.json");
ProcessFile("CalculateGPAByDiscipline.json", "CalculateGPAByDiscipline_out.json");